In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        (os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
# Run this cell once at top. It installs MONAI, timm and skimage etc.
!pip install -q monai==1.3.0
!pip install -q timm
!pip install -q scikit-image
!pip install -q opencv-python-headless


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 23.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 74.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 83.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 46.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 33.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 9.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 91.9 MB/s eta 0:00:00:00:0100:01
ERROR: pip's depen

In [3]:
import os, sys, glob, shutil, random, time
from pathlib import Path
import numpy as np
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt

import cv2
from skimage import filters, morphology, exposure
from sklearn.cluster import KMeans

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import AdamW
from torch.utils.data import Dataset, DataLoader

from monai.networks.nets import SwinUNETR
from monai.transforms import (
    Compose, LoadImage, EnsureChannelFirst, Resize, ScaleIntensity, ToTensor
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)


2025-12-10 18:08:02.716619: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1765390082.883824      47 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1765390082.935534      47 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

Device: cuda


In [4]:
# Set the path to the lung images folder inside the Kaggle dataset
DATA_DIR = "/kaggle/input/lung-and-colon-cancer-histopathological-images/lung_colon_image_set/lung_image_sets"
assert os.path.exists(DATA_DIR), f"Dataset folder not found at {DATA_DIR}. Attach dataset to the notebook."

# show folders and counts
folders = sorted([d for d in os.listdir(DATA_DIR) if os.path.isdir(os.path.join(DATA_DIR, d))])
print("Detected folders:", folders)
for f in folders:
    p = os.path.join(DATA_DIR, f)
    imgs = len([x for x in os.listdir(p) if x.lower().endswith(('.jpg','.png','.jpeg'))])
    print(f, "->", imgs)


Detected folders: ['lung_aca', 'lung_n', 'lung_scc']
lung_aca -> 5000
lung_n -> 5000
lung_scc -> 5000


In [5]:
WORKDIR = "/kaggle/working/lung_work"
os.makedirs(WORKDIR, exist_ok=True)
IMG_DIR = os.path.join(WORKDIR, "images")
MASK_DIR = os.path.join(WORKDIR, "pseudo_masks")
os.makedirs(IMG_DIR, exist_ok=True)
os.makedirs(MASK_DIR, exist_ok=True)

# copy lung folders (only) into working images folder (so original input untouched)
lung_dirs = [d for d in folders if 'lung' in d.lower()]
print("lung_dirs:", lung_dirs)
count=0
for d in lung_dirs:
    src = os.path.join(DATA_DIR, d)
    for f in os.listdir(src):
        if f.lower().endswith(('.jpg','.png','.jpeg')):
            dst = os.path.join(IMG_DIR, f)
            if not os.path.exists(dst):
                shutil.copy2(os.path.join(src, f), dst)
                count += 1
print("Copied images to", IMG_DIR, "count =", count)
print("Masks will be saved to", MASK_DIR)


lung_dirs: ['lung_aca', 'lung_n', 'lung_scc']
Copied images to /kaggle/working/lung_work/images count = 15000
Masks will be saved to /kaggle/working/lung_work/pseudo_masks


In [6]:
# This function tries a pipeline:
# 1) Convert to HSV - threshold on saturation/value with Otsu
# 2) If fails (too small region), fallback to KMeans clustering (k=2) on RGB
# 3) Postprocess: median blur, remove small objects, closing
# 4) Save mask as uint8 (0/255)

def generate_mask_general(img_path, min_area_px=500):
    img_bgr = cv2.imread(img_path)
    if img_bgr is None:
        raise ValueError("Cannot read image: " + img_path)
    img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)
    h, w = img_rgb.shape[:2]

    # 1) HSV + Otsu on Saturation or Value
    hsv = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2HSV)
    s = hsv[:,:,1]
    v = hsv[:,:,2]

    # try Otsu on saturation
    try:
        thr_s = filters.threshold_otsu(s)
        mask_s = (s > thr_s).astype(np.uint8)
    except Exception:
        mask_s = np.zeros((h,w), dtype=np.uint8)

    # try Otsu on value
    try:
        thr_v = filters.threshold_otsu(v)
        mask_v = (v < thr_v).astype(np.uint8)  # darker tissue often corresponds to lesion
    except Exception:
        mask_v = np.zeros((h,w), dtype=np.uint8)

    # combine heuristics
    mask = (mask_s | mask_v).astype(np.uint8)

    # morphological cleanup
    mask = morphology.remove_small_objects(mask.astype(bool), min_size=min_area_px).astype(np.uint8)
    mask = morphology.binary_closing(mask, morphology.disk(5)).astype(np.uint8)
    mask = mask.astype(np.uint8)

    # if mask too small or nearly empty, fallback to kmeans on RGB colors
    if mask.sum() < 0.01 * h * w:  # less than 1% pixels
        flat = img_rgb.reshape(-1,3).astype(np.float32)
        kmeans = KMeans(n_clusters=2, random_state=42).fit(flat)
        labels = kmeans.labels_.reshape(h,w)
        # choose cluster with smaller mean brightness as probable tissue region
        cluster_means = [flat[kmeans.labels_==i].mean() if np.any(kmeans.labels_==i) else 255 for i in range(2)]
        chosen = int(np.argmin(cluster_means))
        mask = (labels == chosen).astype(np.uint8)
        mask = morphology.remove_small_objects(mask.astype(bool), min_size=min_area_px).astype(np.uint8)
        mask = morphology.binary_closing(mask, morphology.disk(5)).astype(np.uint8)
        mask = mask.astype(np.uint8)

    # final smoothing and convert to 0/255
    mask_uint8 = (mask*255).astype(np.uint8)
    mask_uint8 = cv2.medianBlur(mask_uint8, 5)
    # ensure non-empty: if still empty, mark whole image (fallback)
    if mask_uint8.sum() == 0:
        mask_uint8[:] = 0  # keep empty (we don't want false positive), but you can set to 255 if you prefer
    return mask_uint8


In [ ]:
# iterate images and save masks
img_paths = sorted(glob.glob(os.path.join(IMG_DIR, "*.*")))
print("Total images to process:", len(img_paths))

# Optionally process only first N for quick test; set N=None to do all
N = None
if N is not None:
    img_paths = img_paths[:N]

for i, p in enumerate(img_paths):
    try:
        mask = generate_mask_general(p, min_area_px=400)
        out_name = os.path.basename(p)
        cv2.imwrite(os.path.join(MASK_DIR, out_name), mask)
    except Exception as e:
        print("Error for", p, ":", str(e))
    if (i+1) % 200 == 0:
        print("Processed", i+1, "images")

print("Saved masks to", MASK_DIR, " count:", len(os.listdir(MASK_DIR)))


Total images to process: 15000
Processed 200 images
Processed 400 images
Processed 600 images
Processed 800 images
Processed 1000 images
Processed 1200 images
Processed 1400 images
Processed 1600 images
Processed 1800 images
Processed 2000 images
Processed 2200 images
Processed 2400 images
Processed 2600 images
Processed 2800 images
Processed 3000 images
Processed 3200 images
Processed 3400 images
Processed 3600 images
Processed 3800 images
Processed 4000 images


In [ ]:
# display a few images and corresponding masks
sample = img_paths[:6]
plt.figure(figsize=(12,8))
for i, p in enumerate(sample):
    img = Image.open(p).convert("RGB")
    mask_p = os.path.join(MASK_DIR, os.path.basename(p))
    mask = Image.open(mask_p).convert("L") if os.path.exists(mask_p) else Image.new('L', img.size)
    plt.subplot(3,4,2*i+1)
    plt.imshow(img); plt.title("Image"); plt.axis('off')
    plt.subplot(3,4,2*i+2)
    plt.imshow(img); plt.imshow(mask, cmap='jet', alpha=0.45); plt.title("Overlay mask"); plt.axis('off')
plt.tight_layout()


In [ ]:
# Build dataframe of (image, mask)
rows=[]
for img_path in sorted(glob.glob(os.path.join(IMG_DIR, "*.*"))):
    mask_path = os.path.join(MASK_DIR, os.path.basename(img_path))
    if os.path.exists(mask_path):
        rows.append({"image": img_path, "mask": mask_path})
df = pd.DataFrame(rows)
print("Total pairs:", len(df))
df = df.sample(frac=1, random_state=42).reset_index(drop=True)  # shuffle

train_df, val_df = train_test_split(df, test_size=0.15, random_state=42)
train_df = train_df.reset_index(drop=True)
val_df = val_df.reset_index(drop=True)
print("Train:", len(train_df), "Val:", len(val_df))


In [ ]:
class SegDatasetTorch(Dataset):
    def __init__(self, df, img_size=(256,256)):
        self.df = df.reset_index(drop=True)
        self.img_size = img_size
        # using MONAI transforms for simplicity
        self.transform_img = Compose([
            LoadImage(image_only=True),
            EnsureChannelFirst(),
            Resize(spatial_size=self.img_size),
            ScaleIntensity(),
            ToTensor()
        ])
        self.transform_mask = Compose([
            LoadImage(image_only=True),
            EnsureChannelFirst(),
            Resize(spatial_size=self.img_size),
            ToTensor()
        ])
    def __len__(self):
        return len(self.df)
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img = self.transform_img(row.image)    # shape [C,H,W], float, scaled
        mask = self.transform_mask(row.mask)  # [1,H,W]
        # ensure mask is binary 0/1
        mask = (mask > 0.5).float()
        return img.float(), mask.float()

# create dataloaders
IMG_SIZE = (256,256)
train_ds = SegDatasetTorch(train_df, img_size=IMG_SIZE)
val_ds = SegDatasetTorch(val_df, img_size=IMG_SIZE)
train_loader = DataLoader(train_ds, batch_size=8, shuffle=True, num_workers=2, pin_memory=True)
val_loader = DataLoader(val_ds, batch_size=8, shuffle=False, num_workers=2, pin_memory=True)
print("Dataloaders ready. Train batches:", len(train_loader), "Val batches:", len(val_loader))


In [ ]:
# Build SwinUNETR with MONAI
# in_channels = 3 for RGB, out_channels = 1 (binary mask)
model = SwinUNETR(
    img_size=IMG_SIZE,
    in_channels=3,
    out_channels=1,
    feature_size=48,   # reduce feature_size if GPU memory low
    drop_rate=0.0,
).to(device)

# losses and optimizer
bce_loss = nn.BCEWithLogitsLoss()
def dice_loss_logits(logits, target, smooth=1e-6):
    probs = torch.sigmoid(logits)
    inter = (probs * target).sum(dim=(2,3))
    denom = probs.sum(dim=(2,3)) + target.sum(dim=(2,3))
    loss = 1 - ((2*inter + smooth) / (denom + smooth))
    return loss.mean()

optimizer = AdamW(model.parameters(), lr=1e-4, weight_decay=1e-5)
print("Model params (millions):", sum(p.numel() for p in model.parameters())/1e6)


In [ ]:
EPOCHS = 12
best_val_dice = 0.0
save_path = "/kaggle/working/swin_unetr_best.pth"

for epoch in range(1, EPOCHS+1):
    model.train()
    train_loss = 0.0
    train_batches = 0
    for imgs, masks in train_loader:
        imgs = imgs.to(device)
        masks = masks.to(device)
        preds = model(imgs)  # logits [B,1,H,W]
        if preds.shape != masks.shape:
            preds = F.interpolate(preds, size=masks.shape[-2:], mode='bilinear', align_corners=False)
        loss_bce = bce_loss(preds, masks)
        loss_dice = dice_loss_logits(preds, masks)
        loss = loss_bce + loss_dice
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
        train_batches += 1
    train_loss = train_loss / max(1, train_batches)

    # validation
    model.eval()
    val_losses = 0.0
    val_batches = 0
    dice_scores = []
    with torch.no_grad():
        for imgs, masks in val_loader:
            imgs = imgs.to(device)
            masks = masks.to(device)
            preds = model(imgs)
            if preds.shape != masks.shape:
                preds = F.interpolate(preds, size=masks.shape[-2:], mode='bilinear', align_corners=False)
            loss_bce = bce_loss(preds, masks)
            loss_dice = dice_loss_logits(preds, masks)
            val_losses += (loss_bce.item() + loss_dice.item())
            val_batches += 1
            # compute dice per batch
            probs = torch.sigmoid(preds)
            pred_bin = (probs > 0.5).float()
            inter = (pred_bin * masks).sum(dim=(2,3))
            denom = pred_bin.sum(dim=(2,3)) + masks.sum(dim=(2,3)) + 1e-6
            dice_batch = (2*inter / denom).mean().item()
            dice_scores.append(dice_batch)
    val_loss = val_losses / max(1, val_batches)
    val_dice = np.mean(dice_scores) if dice_scores else 0.0

    print(f"Epoch {epoch}/{EPOCHS} TrainLoss:{train_loss:.4f} ValLoss:{val_loss:.4f} ValDice:{val_dice:.4f}")

    # save best
    if val_dice > best_val_dice:
        best_val_dice = val_dice
        torch.save(model.state_dict(), save_path)
        print("Saved best model with ValDice:", best_val_dice)


In [ ]:
# load best model (if saved) and show some predictions
if os.path.exists("/kaggle/working/swin_unetr_best.pth"):
    model.load_state_dict(torch.load("/kaggle/working/swin_unetr_best.pth", map_location=device))
model.eval()

# get first validation batch
imgs, masks = next(iter(val_loader))
imgs = imgs.to(device)
with torch.no_grad():
    preds = model(imgs)
    probs = torch.sigmoid(preds).cpu().numpy()
imgs = imgs.cpu().numpy()
masks = masks.cpu().numpy()

n_show = min(6, imgs.shape[0])
plt.figure(figsize=(12,4*n_show//3 + 2))
for i in range(n_show):
    img = np.transpose(imgs[i], (1,2,0))
    # undo intensity scaling (ScaleIntensity scales to 0-1 approx)
    img = np.clip(img, 0, 1)
    pred_mask = probs[i,0]
    gt_mask = masks[i,0]
    plt.subplot(n_show,3,3*i+1); plt.imshow(img); plt.axis('off'); plt.title('Image')
    plt.subplot(n_show,3,3*i+2); plt.imshow(img); plt.imshow(pred_mask, alpha=0.45); plt.axis('off'); plt.title('Pred overlay')
    plt.subplot(n_show,3,3*i+3); plt.imshow(gt_mask, cmap='gray'); plt.axis('off'); plt.title('Pseudo GT')
plt.tight_layout()


In [ ]:
print("Best model saved at:", "/kaggle/working/swin_unetr_best.pth")

def predict_on_image(model, img_path, out_size=(256,256), thr=0.5):
    model.eval()
    img = np.array(Image.open(img_path).convert("RGB"))
    # resize
    img_res = cv2.resize(img, out_size[::-1])  # (W,H)
    img_norm = img_res.astype(np.float32) / 255.0
    # MONAI transforms: channel first, tensor
    inp = torch.tensor(img_norm.transpose(2,0,1)[None]).float().to(device)
    with torch.no_grad():
        logits = model(inp)
        probs = torch.sigmoid(logits)[0,0].cpu().numpy()
    mask = (probs > thr).astype(np.uint8) * 255
    return img_res, probs, mask

# Example usage:
# img_res, probs, mask = predict_on_image(model, train_df.iloc[0]['image'])
# plt.imshow(img_res); plt.imshow(mask, alpha=0.4)
